# Análise - O Oscar na Última Década

## Integrantes:

### João Victor Soares Silva | Mat.: 20210027300
### Marília Gabriela Toscano da Silva | Mat.:
### Matheus Victor Alves Braga Maciel | Mat.:

# Introdução

///Texto da introdução e motivação

## Objetivos

### Objetivos Gerais
///Falar sobre explorar os dados e descobrir algo

### Objetivos Específicos
///Falar das perguntas

# Dados Usados

///Descrever os datasets

Bibliotecas utilizadas:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

# Pré-Processamento

///Limpeza dos dataframes

# Quais as características mais frequentes entre os filmes indicados à premiação?

# Dentre os indicados, qual o perfil dos filmes que foram melhor aceitos pelo público e pela crítica?

# Qual o melhor parâmetro, entre os sites estudados, para determinar as chances de vitória de um filme na premiação?

# Conclusão

///Resumir as respostas, destacar sua importância, apontar limitações, trabalhos futuros e melhorias.